<h2>Don't Overfit </h2>

<h2>Problem Statement </h2>

- Donot Overfit 2 is a unique problem statement where we are provided with only 250 training samples and 19750 test samples.
- The Objective of the problem is not to overfit with this train data and generalize well with our test data samples.
- The data set consists of 300 continuous random variables each standardized with mean centered to zero and variance 1.

<h2> Performance Metrics Used </h2>

- The Problem uses ROC AUC SCORE as the metric to measure the model performance

In [103]:
import numpy as np
import pandas as pd
from scipy import stats
import sklearn
import warnings
warnings.filterwarnings('ignore')
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import RepeatedStratifiedKFold
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
pd.options.mode.chained_assignment = None   # default='warn'
from mlxtend.classifier import StackingClassifier
from sklearn.linear_model import Lasso
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from scipy.stats import randint as sp_randint
from scipy.stats import uniform
from scipy import stats
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

<h1>Getting the Data into Data Frame </h1>

In [109]:
train_data=pd.read_csv("train.csv")
test_data=pd.read_csv("test.csv")

<h5>Data Split</h5>

In [110]:
train_data["target"]=train_data["target"].astype(int)
train_data=train_data.drop("id",axis=1)
X=train_data.drop("target",axis=1)
Y=train_data['target']
test=test_data.drop("id",axis=1)

In [111]:
def simple_feature_engg(data):
  data["mean"]=np.mean(data,axis=1)
  data["sum"]=np.sum(data,axis=1)
  data["std_dev"]=np.std(data,axis=1)
  return data

In [112]:
X_fe=simple_feature_engg(X)

In [113]:
test_fe=simple_feature_engg(test_fe)

In [107]:
from sklearn.preprocessing import MinMaxScaler
def normalize(data,test_data):
  scaler=MinMaxScaler()
  scaler=scaler.fit(data)
  trans_data=scaler.transform(data)
  test_trans_data=scaler.transform(test_data)
  return((pd.DataFrame(trans_data,columns=data.columns),pd.DataFrame(test_trans_data,columns=data.columns)))

In [114]:
X_scaled,test_scaled=normalize(X_fe,test_fe)

In [115]:
X_scaled.head(1)

,0,1,2,3,4,5,6,7,8,9,...,293,294,295,296,297,298,299,mean,sum,std_dev
0,0.454564,0.952523,0.538081,0.340156,0.708798,0.423909,0.496197,0.591415,0.098864,0.786842,...,0.358949,0.718121,0.157182,0.627768,0.437245,0.630322,0.388295,0.442476,0.442476,0.094353


In [116]:
X_scaled.shape

(250, 303)

In [117]:
def model_with_parameters(model_name):

  if model_name=="KNN":
    model=KNeighborsClassifier()
    param_= {'n_neighbors':np.arange(3,45,2).tolist(),'algorithm':['kdtree','brute']}

  elif model_name=="LR":
    model=SGDClassifier(loss='log',class_weight='balanced',n_jobs=-1)
    param_={"penalty":["l1","l2","elasticnet"],
        "alpha":np.arange(0.1,0.9,0.01),
        "l1_ratio":np.arange(0.1,0.9,0.05)
      }

  elif model_name== "SVM":
    model=SGDClassifier(loss='hinge',class_weight='balanced',n_jobs=-1)
    param_={"penalty":["l1","l2","elasticnet"],
        "alpha":np.arange(0.1,0.9,0.01),
        "l1_ratio":np.arange(0.1,0.9,0.05)}
      
  elif model_name == "GNB":
    model=GaussianNB(priors=[0.5,0.5])
    param_=""

  elif model_name == "RF":
    model= RandomForestClassifier(random_state=42)
    param_={'n_estimators':[10,20,30,40,50,100,200,300,400],'max_depth':[2,3,5,7]}

  elif model_name == "XGB":
    model=XGBClassifier(scale_pos_weight=0.5)
    param_={"max_depth":[2,3,5,7],'n_estimators':[10,20,50,100,200,300]}

  return (model,param_) 

<h2> Solution-2 Stacking Classifier and Rigorous Feature Selection </h2>

In [118]:
def rskf_func(n_splits,n_repeats): #Creating a function for repeated stratified K-Fold Object
    rskf_var=RepeatedStratifiedKFold(n_splits=n_splits,n_repeats=n_repeats)
    return rskf_var

In [30]:
def model_training(train_data,Y,test_data,model_name):
  iter_=1
  cnt=0
  test_prediction=np.zeros(len(test_data))
  rskf=rskf_func(5,2)

  for train_index,validation_index in rskf.split(train_data,Y):
    print("Iter:",iter_)
    X_train,X_CV=train_data.loc[train_index],train_data.loc[validation_index]
    Y_train,Y_CV=Y.iloc[train_index],Y.iloc[validation_index]


    feature_selector_model=Lasso()
    grid_par={"alpha":[0.022, 0.021, 0.02, 0.019, 0.023, 0.024, 0.025, 0.026, 0.027, 0.029, 0.031]}
    gridcv=RandomizedSearchCV(feature_selector_model,grid_par,cv=10,scoring='roc_auc',n_jobs=-1,verbose=0)
    gridcv.fit(X_train,Y_train)


    sfs = SFS(gridcv.best_estimator_,k_features=(10, 15),forward=True,floating=False,scoring='roc_auc',verbose=1,n_jobs=-1)
    sfs.fit(X_train.values,Y_train.values)


    X_train_imp=sfs.transform(X_train)
    X_Cv_imp=sfs.transform(X_CV)
    test_imp=sfs.transform(test_data)

    model,param=model_with_parameters(model_name)
    
    if model_name == "GNB":
        
        model.fit(X_train_imp, Y_train)

        calib_classifier=CalibratedClassifierCV(model,cv=5,method='sigmoid')
        calib_classifier.fit(X_train_imp,Y_train)
        
    else:
        
        grid =RandomizedSearchCV(estimator=model,param_distributions=param,cv=5,scoring='roc_auc',n_jobs=-1,verbose=1)
        grid.fit(X_train_imp, Y_train)

        calib_classifier=CalibratedClassifierCV(grid.best_estimator_,cv=5,method='sigmoid')
        calib_classifier.fit(X_train_imp,Y_train)
  
    y_cv_pred=calib_classifier.predict_proba(X_Cv_imp)[:,1]

    roc_cv=roc_auc_score(Y_CV.values,y_cv_pred)
    
    if(roc_cv > 0.75):
        print("<--------Model is performing well------->")
        test_prediction+=calib_classifier.predict_proba(test_imp)[:,1]
        cnt+=1
    else:
        print("<------The Model is not performing as expected in this iteration--------->")
    iter_+=1

  final_prediction=test_prediction * (1./cnt)

  return final_prediction



In [31]:
model_list=["KNN","LR","SVM","GNB","RF","XGB"]
pred_dict={}
for model in tqdm(model_list):
  prediction=model_training(X_scaled,Y,test_scaled,model)
  pred_dict[model]=prediction

  0%|                                                                                            | 0/6 [00:00<?, ?it/s]

Iter: 1


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 303 out of 303 | elapsed:    0.4s finished
Features: 1/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 302 out of 302 | elapsed:    0.4s finished
Features: 2/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 301 out of 301 | elapsed:    0.4s finished
Features: 3/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.4s finished
Features: 4/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 

Fitting 5 folds for each of 10 candidates, totalling 50 fits
<------The Model is not performing as expected in this iteration--------->
Iter: 2


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 303 out of 303 | elapsed:    0.4s finished
Features: 1/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 302 out of 302 | elapsed:    0.4s finished
Features: 2/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 301 out of 301 | elapsed:    0.4s finished
Features: 3/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.4s finished
Features: 4/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 

...

<------The Model is not performing as expected in this iteration--------->
Iter: 10


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 303 out of 303 | elapsed:    0.4s finished
Features: 1/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 302 out of 302 | elapsed:    0.4s finished
Features: 2/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 301 out of 301 | elapsed:    0.4s finished
Features: 3/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.0s
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:    0.4s finished
Features: 4/15[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed:    0.2s finished
100%|████████████████████████████████████████████████████████████████████████████████████| 6/6 [08:36<00:00, 86.01s/it]

<--------Model is performing well------->


In [131]:
test_scaled.shape

(19750, 303)

In [145]:
from sklearn.model_selection import GridSearchCV
def stacking_classifier(data,te_scaled,Y):
    iter_=1
    test_val=np.zeros(len(te_scaled))
    cnt=0
    rskf=rskf_func(10,4)
    clf1,param1=model_with_parameters("LR")
    clf2,param2=model_with_parameters("SVM")
    clf3,param3=model_with_parameters("RF")
    clf4,param4=model_with_parameters("XGB")
    for train_index,valid_index in rskf.split(data,Y):
          #Gets 19 chunks of splitted data for train and 1 chunk for the validation out of 20 chunks
          #Each chunk can be used for validation once.Hence 20 iterations for 20 chunks
          #And we are repeating the process for 20 times.So 400 iterations in total
        X_train,X_CV=X.loc[train_index],X.loc[valid_index]
        Y_train,Y_CV=Y.loc[train_index],Y.loc[valid_index]
        feature_selector_model=Lasso()
        grid_par={"alpha":[0.022, 0.021, 0.02, 0.019, 0.023, 0.024, 0.025, 0.026, 0.027, 0.029, 0.031]}
        gridcv=RandomizedSearchCV(feature_selector_model,grid_par,cv=10,scoring='roc_auc',n_jobs=-1,verbose=1)
        gridcv.fit(X_train,Y_train)
        sfs = SFS(gridcv.best_estimator_,k_features=(10, 20),forward=True,floating=True,scoring='roc_auc',verbose=1,n_jobs=-1)
        sfs.fit(X_train,Y_train)
        X_train_imp=sfs.transform(X_train)
        X_Cv_imp=sfs.transform(X_CV)
        test_imp=sfs.transform(te_scaled)

    

        grid_model_1=GridSearchCV(clf1,param1,cv=10,scoring='roc_auc',n_jobs=-1,verbose=0)
        grid_model_1.fit(X_train_imp,Y_train)
        clf_1_calib=CalibratedClassifierCV(grid_model_1.best_estimator_,cv=10,method='sigmoid')
        clf_1_calib.fit(X_train_imp,Y_train)

        grid_model_2=GridSearchCV(clf2,param2,cv=5,scoring='roc_auc',n_jobs=-1,verbose=0)
        grid_model_2.fit(X_train_imp,Y_train)
        clf_2_calib=CalibratedClassifierCV(grid_model_2.best_estimator_,cv=10,method='sigmoid')
        clf_2_calib.fit(X_train_imp,Y_train)

        grid_model_3=GridSearchCV(clf3,param3,cv=5,scoring='roc_auc',n_jobs=-1,verbose=0)
        grid_model_3.fit(X_train_imp,Y_train)
        clf_3_calib=CalibratedClassifierCV(grid_model_3.best_estimator_,cv=10,method='sigmoid')
        clf_3_calib.fit(X_train_imp,Y_train)

        grid_model_4=GridSearchCV(clf4,param4,cv=5,scoring='roc_auc',n_jobs=-1,verbose=0)
        grid_model_4.fit(X_train_imp,Y_train)
        clf_4_calib=CalibratedClassifierCV(grid_model_4.best_estimator_,cv=10,method='sigmoid')
        clf_4_calib.fit(X_train_imp,Y_train)

        sclf=StackingClassifier(classifiers=[clf_1_calib,clf_2_calib,clf_3_calib,clf_4_calib],meta_classifier=clf_3_calib,use_probas=True)

        sclf.fit(X_train_imp,Y_train)

        score=roc_auc_score(Y_CV.values,sclf.predict_proba(X_Cv_imp)[:,1])

        if score > 0.9:
            print("Model is ok for this iteration")
            test_val+=sclf.predict_proba(test_imp)[:,1]
            cnt+=1
        else:
            print("Skipping this iteration")
            
        iter_=iter_ + 1
            

    final_stack_prediction=test_val* (1./cnt)

    return final_stack_prediction
      

In [146]:
stack_sfs_pred=stacking_classifier(X_scaled,test_scaled,Y)

Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 303 out of 303 | elapsed:    0.5s finished
Features: 1/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 302 out of 302 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 301 out of 301 | elapsed:    0.5

Skipping this iteration
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  85 out of 100 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 303 out of 303 | elapsed:    0.6s finished
Features: 1/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 302 out of 302 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks    

Skipping this iteration
Fitting 10 folds for each of 10 candidates, totalling 100 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 303 out of 303 | elapsed:    0.5s finished
Features: 1/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 302 out of 302 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 301 out of 301 | elapsed:    0.5

[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 285 out of 285 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 out of  18 | elapsed:    0.0s finished
Features: 19/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 284 out of 284 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  19 out of  19 | elapsed:    0.0s finished
Features: 20/20

Skipping this iteration
Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 303 out of 303 | elapsed:    0.6s finished
Features: 1/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 302 out of 302 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.0s finished
Features: 2/20[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 301 out of 301 | elapsed:    0.5

Skipping this iteration


In [147]:
stack_sfs={}
stack_sfs["Stack"] = stack_sfs_pred

In [56]:
def final_submission_csv(name): #Getting csv for kaggle submission
    sub_df=pd.read_csv("sample_submission.csv")
    for k,v in name.items():
      final_df=pd.concat([sub_df["id"],pd.DataFrame(v)],axis=1)
      final_df.columns=["id","target"]
      final_df.to_csv(k + "_SFS" + ".csv",index=False)

In [37]:
final_submission_csv(pred_dict)

In [148]:
final_submission_csv(stack_sfs)

<h2>Final Score </h2>

<b>

- With Stacked Model we get a Private Score of 0.825 and Public Score of 0.838

</b>

<h2>Conclusion </h2>

- Simple Logistic Regression with K-Fold Cross Validation we got 80 % ROC 
- More Complex Stacked Classifer and Feature Selection Techniques help us achieve 83% ROC
- Without LB Probing we could get a good classifier which separates the data well in  Private LB 

<h2>References </h2>

- https://www.kaggle.com/featureblind/robust-lasso-patches-with-rfe-gs
- https://www.kaggle.com/rafjaa/dealing-with-very-small-datasets
- https://www.kaggle.com/iavinas/simple-short-solution-don-t-overfit-0-848
- https://www.appliedaicourse.com/lecture/11/applied-machine-learning-online-course/3096/stacking-models/4/module-4-machine-learning-ii-supervised-learning-models